In [1]:
import os
import pandas as pd
import gmaps
import googlemaps as g
import yaml
import numpy
import matplotlib.pyplot as plt
from IPython.display import Image
import foursquare



%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.notebook_repr_html', True)

# Loading dataset
df = pd.read_csv(os.path.join('..', 'datasets', 'foursquare_idmapping.csv'))
# df = df.set_index(id)
df.drop(['Unnamed: 0'],inplace = True, axis = 1)
df.head()

,business_name,location,inspection_score,foursquare_id,foursquare_name
0,Tiramisu Kitchen,"(37.791116, -122.403816)",87.400000,4ae289b9f964a520e78e21e3,Cafe Tiramisu
1,Nrgize Lifestyle Cafe,"(37.786848, -122.421547)",94.000000,4c1a7695eac020a1105043c2,Nrgize Lifestyle Cafe
2,OMNI S.F. Hotel - 2nd Floor Pantry,"(37.792888, -122.403135)",96.333333,NaN,NaN
3,Norman's Ice Cream and Freezes,"(37.807155, -122.419004)",87.750000,4bb7d9273db7b71353aa209a,Norman's Ice Cream & Freezes
4,CHARLIE'S DELI CAFE,"(37.747114, -122.413641)",83.200000,4ab90c8bf964a520d27d20e3,Charlie's Deli Cafe


In [2]:
# Applying foursquare credentials
with open('foursquare.yaml', 'r') as f:
    fs_credentials = yaml.load(f)

fs_client_id = fs_credentials['client-id']
fs_client_secret = fs_credentials['client-secret']



In [3]:
client = foursquare.Foursquare(client_id=fs_client_id, client_secret=fs_client_secret)

In [4]:
# df.dropna(subset = ['foursquare_id'], inplace = True)
len(df)

5568

In [47]:
def foursquare_attributes(id):
    restaurant = client.venues(id)
    if restaurant['venue']:
        r = restaurant['venue']
        return {
            'foursquare_id': id,
            'price': r.get('price', {}).get('tier'),
            'hasMenu': r.get('hasMenu') != None,
            'rating': r.get('rating'),
            'ratingSignals': r.get('ratingSignals'), 
            'photos': r.get('photos').get('count'),
#             'hasReservations': r.get('attributes').get('groups').'type'.'reservations'.get('displayValue') != None,
            'tipCount': r.get('stats').get('tipCount')
        }
    
r = foursquare_attributes(df.foursquare_id[0])

In [48]:
{}.get('a')

In [49]:
foursquare_rows = [
    foursquare_attributes(df.foursquare_id[i]) 
    for i in range(0, len(df)) if not(pd.isnull(df.foursquare_id[i]))
]
# foursquare_rows

In [50]:
foursquare_df = pd.DataFrame(foursquare_rows)
foursquare_df

,foursquare_id,hasMenu,photos,price,rating,ratingSignals,tipCount
0,4ae289b9f964a520e78e21e3,True,79,3.0,6.3,56.0,18
1,4c1a7695eac020a1105043c2,False,1,1.0,NaN,NaN,1
2,4bb7d9273db7b71353aa209a,True,53,1.0,8.9,71.0,24
3,4ab90c8bf964a520d27d20e3,False,41,1.0,7.4,33.0,10
4,3fd66200f964a52019f11ee3,True,167,1.0,8.8,190.0,82
5,4c1f9a12b306c92885ad68b7,True,133,3.0,8.2,195.0,72
6,4dbcbee46e810768bf678f0f,False,0,NaN,NaN,NaN,0
7,44d756a9f964a52068361fe3,False,62,1.0,6.1,71.0,30
8,4a6f5d0ef964a520fdd51fe3,True,92,1.0,8.9,353.0,83


In [51]:
# df.join(foursquare_df, on='foursquare_id', lsuffix='l_', rsuffix='r_')
df.merge(foursquare_df, on='foursquare_id')

,business_name,location,inspection_score,foursquare_id,foursquare_name,hasMenu,photos,price,rating,ratingSignals,tipCount
0,Tiramisu Kitchen,"(37.791116, -122.403816)",87.400000,4ae289b9f964a520e78e21e3,Cafe Tiramisu,True,79,3.0,6.3,56.0,18
1,Nrgize Lifestyle Cafe,"(37.786848, -122.421547)",94.000000,4c1a7695eac020a1105043c2,Nrgize Lifestyle Cafe,False,1,1.0,NaN,NaN,1
2,Norman's Ice Cream and Freezes,"(37.807155, -122.419004)",87.750000,4bb7d9273db7b71353aa209a,Norman's Ice Cream & Freezes,True,53,1.0,8.9,71.0,24
3,CHARLIE'S DELI CAFE,"(37.747114, -122.413641)",83.200000,4ab90c8bf964a520d27d20e3,Charlie's Deli Cafe,False,41,1.0,7.4,33.0,10
4,ART'S CAFE,"(37.764013, -122.465749)",92.500000,3fd66200f964a52019f11ee3,Art's Cafe,True,167,1.0,8.8,190.0,82
5,SUSHI ZONE,"(37.771437, -122.423892)",82.500000,4c1f9a12b306c92885ad68b7,Sushi Zone,True,133,3.0,8.2,195.0,72
6,RHODA GOLDMAN PLAZA,"(37.784626, -122.437734)",94.666667,4dbcbee46e810768bf678f0f,Rhoda Goldman Plaza,False,0,NaN,NaN,NaN,0
7,CAFE X + O,"(37.742325, -122.426476)",91.333333,44d756a9f964a52068361fe3,Cafe XO,False,62,1.0,6.1,71.0,30
8,Oasis Grill,"(37.794483, -122.396584)",75.000000,4a6f5d0ef964a520fdd51fe3,Oasis Grill,True,92,1.0,8.9,353.0,83
